In [71]:
import pandas as pd
import numpy as np

def load_and_combine_data(directory_path):
    """Carrega X e y, limpa o cabeçalho e retorna X_train, y_train, X_test, y_test."""
    
    try:
        # Carregar X: assumindo cabeçalho correto
        X_train = pd.read_csv(f"{directory_path}/X_train.csv")
        X_test = pd.read_csv(f"{directory_path}/X_test.csv")
        
        # Carregar Y: Pulando o cabeçalho 'Y' dentro do arquivo (skiprows=1)
        y_train = pd.read_csv(f"{directory_path}/y_train.csv", 
                              header=None, 
                              names=['Y'], 
                              skiprows=1)
                             
        y_test = pd.read_csv(f"{directory_path}/y_test.csv", 
                             header=None, 
                             names=['Y'], 
                             skiprows=1)

        # 1. Limpeza e Conversão para Numérico (Essencial após skiprows)
        y_train['Y'] = pd.to_numeric(y_train['Y'].squeeze(), errors='coerce') 
        y_test['Y'] = pd.to_numeric(y_test['Y'].squeeze(), errors='coerce') 

        # 2. Verificar o Alinhamento (Safety Check)
        if len(X_train) != len(y_train) or len(X_test) != len(y_test):
             print(f"ERRO: Desalinhamento de tamanho. Treino: {len(X_train)} vs {len(y_train)}. Teste: {len(X_test)} vs {len(y_test)}.")
             return None, None, None, None # Retorna None se falhar
        
        # 3. Retornar no formato correto para model.fit()
        # y_train e y_test são Series 1D
        return X_train, y_train, X_test, y_test
    
    except FileNotFoundError as e:
        print(f"Erro ao carregar arquivos no caminho {directory_path}: {e}")
        return None, None, None, None

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from scipy.stats import spearmanr
import shap 

SENSITIVE_FEATURE_NAME = 'Q' 

def run_experiment(X_train, y_train, X_test, y_test, baseline_feature_ranking=None):
    """Treina o modelo e calcula métricas usando os arrays X e y separados."""
    
    results = {}
    
    # --- 1. Model Training ---
    # y_train já deve estar no formato 1D (ravel)
    model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # --- 2. Robustness Metrics ---
    results['Overall_Accuracy'] = accuracy_score(y_test, y_pred)
    results['Overall_F1_Score'] = f1_score(y_test, y_pred)
    results['AUROC'] = roc_auc_score(y_test, y_proba)
    
    # --- 3. Fairness Metrics (EOD) ---
    Q_test = X_test[SENSITIVE_FEATURE_NAME]
    
    # Cálculo do True Positive Rate (TPR)
    # Grupo Privilegiado (Q=0)
    TP_P = ((y_test == 1) & (Q_test == 0) & (y_pred == 1)).sum()
    P_pos = ((y_test == 1) & (Q_test == 0)).sum()
    TPR_P = TP_P / P_pos if P_pos > 0 else 0
    
    # Grupo Desprivilegiado (Q=1)
    TP_U = ((y_test == 1) & (Q_test == 1) & (y_pred == 1)).sum()
    U_pos = ((y_test == 1) & (Q_test == 1)).sum()
    TPR_U = TP_U / U_pos if U_pos > 0 else 0

    results['Equal_Opportunity_Difference'] = TPR_P - TPR_U
    
    # --- 4. Explainability Metrics (SHAP Stability) ---
    explainer = shap.Explainer(model, X_train)
    shap_values = explainer(X_test.sample(n=min(500, len(X_test)), random_state=42)) 
    
    mean_abs_shap = pd.Series(np.abs(shap_values.values).mean(axis=0), index=X_test.columns)
    current_feature_ranking = mean_abs_shap.sort_values(ascending=False).rank(method='first')

    results['Feature_Importance_Ranking'] = current_feature_ranking.to_dict()

    if baseline_feature_ranking is not None:
        I_base = pd.Series(baseline_feature_ranking)
        I_current = current_feature_ranking
        common_features = I_base.index.intersection(I_current.index)
        
        rho, _ = spearmanr(I_base.loc[common_features].values, 
                           I_current.loc[common_features].values)
        results['SHAP_Rank_Stability'] = rho
    else:
        results['SHAP_Rank_Stability'] = 1.0 

    return results

In [73]:
# --- LOOP PRINCIPAL DE EXECUÇÃO ---

# Lista de diretórios de experimentos (substitua pelos seus caminhos reais)
experiment_directories = [
    'datasets/baseline',
]
for i in [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7, 0.8,0.9]:
    experiment_directories.append(f'datasets/bias_lq_{i}')
    #experiment_directories.append(f'datasets/noise_sy_{i}')
    #experiment_directories.append(f'datasets/imbalance_pu_{i}')

master_results_df = pd.DataFrame()
baseline_ranking = None

print("--- Iniciando Loop de Auditoria ---")

for i, dir_path in enumerate(experiment_directories):
    print(f"\nProcessando: {dir_path.split('/')[-1]}")

    X_train, y_train, X_test, y_test = load_and_combine_data(dir_path)
    print(f"\nPre-split data received:")
    print(f"  X_train shape: {X_train.shape}")
    print(f"  X_test shape: {X_test.shape}")
    print(f"  y_train shape: {y_train.shape}")
    print(f"  y_test shape: {y_test.shape}")
    evaluator, results = fairness_pipeline_presplit(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        sensitive_attribute='A',
        output_csv=f'results/{dir_path.split("/")[-1]}.csv',
        random_state=42
    )
    ress = calculate_overall_fairness_score(results)
    print(f"Overall Fairness Score: {ress:.4f} (lower is better)")

   

--- Iniciando Loop de Auditoria ---

Processando: baseline

Pre-split data received:
  X_train shape: (6, 4)
  X_test shape: (4, 4)
  y_train shape: (6, 1)
  y_test shape: (4, 1)
Training model and evaluating fairness metrics...
FAIRNESS EVALUATION SUMMARY

Model Type: LogisticRegression
Train Accuracy: 1.0000
Test Accuracy: 0.7500
Sensitive Attribute: A
Reference Group: 0

--------------------------------------------------------------------------------
GROUP-SPECIFIC METRICS
--------------------------------------------------------------------------------

Group: 0
  Samples: 2
  TPR: 0.0000
  FPR: 0.0000
  FNR: 1.0000
  Accuracy: 0.5000

Group: 1
  Samples: 2
  TPR: 0.0000
  FPR: 0.0000
  FNR: 0.0000
  Accuracy: 1.0000

--------------------------------------------------------------------------------
FAIRNESS METRICS (vs Reference Group)
--------------------------------------------------------------------------------

1_vs_0:
  ACC (Accuracy Ratio):            2.0000  [Ideal: 1.0]
  PE